# 《Reinforcement Learning》

![](https://github.com/applenob/rl_learn/raw/master/res/rl_book.jpg)

## 目录

- [0.书本介绍](#0.书本介绍)
- [1. Introduction](#1.-Introduction)

## 0.书本介绍

[Reinforcement Learning](https://book.douban.com/subject/2866455/)

作者: Richard S. Sutton / Andrew G. Barto 

内容简介：

```
Reinforcement learning, one of the most active research areas in artificial intelligence, is a computational approach to learning whereby an agent tries to maximize the total amount of reward it receives when interacting with a complex, uncertain environment. 

In Reinforcement Learning, Richard Sutton and Andrew Barto provide a clear and simple account of the key ideas and algorithms of reinforcement learning. Their discussion ranges from the history of the field's intellectual foundations to the most recent developments and applications. 

The only necessary mathematical background is familiarity with elementary concepts of probability.

The book is divided into three parts. 

Part I defines the reinforcement learning problem in terms of Markov decision processes. 

Part II provides basic solution methods: dynamic programming, Monte Carlo methods, and temporal-difference learning. 

Part III presents a unified view of the solution methods and incorporates artificial neural networks, eligibility traces, and planning; the two final chapters present case studies and consider the future of reinforcement learning.
```

其他重要学习资料：

- [WildML的博客](http://www.wildml.com/2016/10/learning-reinforcement-learning/)

- [David Silver’s Reinforcement Learning Course](http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching.html)


## 1. Introduction

**Reinforcement Learning** :强化学习是学习“该如何去做”，如何从某个状态映射到某个行为，来最大化某个数值的奖励信号。

**Exploit vs Explore**: **exploit** 代表利用已有的信息去获得最大的奖励，**explore** 代表去探索一些没有尝试过的行为，去获得可能的更大的奖励。

**强化学习的几个要素**:
- **policy**: 状态到行为的映射，定义agent在某个状态下应该如何采取行为。
- **reward function**: 在某个状态下，agent能够收到的即时反馈。
- **value function**: 衡量在某个状态下，能够获得的长期反馈。
- **modle (of the environment)**: 模型用来模拟环境的行为，给定一个状态和动作，模型可以预测下一个状态和奖励。
   
**RL vs Evolutionary Methods**:
- Evolutionary Methods（遗传算法，具体可以回顾[之前的博客](https://applenob.github.io/ga.html)），直接在policy空间中搜索，并计算最后的得分。通过一代代的进化来找寻最优policy。
- 遗传算法忽略了policy实际上是state到action的映射，它不关注agent和环境的互动，只看最终结果。

**Tic-Tac-Toe（井字棋）**：
![](https://github.com/applenob/rl_learn/raw/master/res/ttt.png)
- 定义policy是任何一种局面下，该如何落子。
- 遗传算法解法：试很多种policy，找到最终胜利的几种，然后结合，更新。
- 强化学习解法：
    - 1.建立一张表格，state_num × 1，代表每个state下，获胜的概率，这个表格就是所谓的**value function**。
    - 2.跟对手下很多局。每次落子的时候，依据是在某个state下，选择所有可能的后继state中，获胜概率最大的（v最大的）。这种方法即贪婪法（Exploit）。偶尔我们也随机选择一些其他的state（Explore）。
    - 3.“back up”后继state的v到当前state上。$V(s)\leftarrow V(s)+\alpha[V(s')-V(s)]$，这就是所谓的差分学习，temporal-difference learning，这么叫是因为$V(s')-V(s)$是两个时间点上的两次估计的差。

## 2. Evaluative Feedback

**k-armed Bandit Problem**：
![](https://github.com/applenob/rl_learn/raw/master/res/bandit.png)